<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

# Imports

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00


In [ ]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "/kaggle/input/bangla-speech-processing-banglaasr/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [ ]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


In [ ]:
mp3_path = "/kaggle/input/narsingdi-train-test/narsingdi/narsingdi_test/rec_23_audio_46.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /kaggle/input/narsingdi-train-test/narsingdi/narsingdi_test/rec_23_audio_46.wav


In [ ]:
import os
test_path = "/kaggle/input/narsingdi-train-test/narsingdi/narsingdi_train/"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)

    #sanity check
    if len(prediction)==0:
        prediction = "<>"

    sentences.append(prediction)



  0%|          | 0/1108 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1108/1108 [1:06:31<00:00,  3.60s/it]


In [ ]:
df1 = pd.DataFrame({"External_ID":ids,"contents":sentences})
df1.head()

,External_ID,contents
0,rec_04_audio_44,"আমার বাইরা থাকতে, আমার বাইডে তবার শেফটায়, পৃথ..."
1,rec_14_audio_55,এর কিনামে সৌরতাহিত্যেও টেস্টেমেদের খেল চীনিয়া...
2,rec_22_audio_69,"সুরুজনীক করছেন, ঢাকতি দাঁপুরদরা, ঢাকতি দিনীকপে..."
3,rec_19_audio_24,আমি উদালি যুটি কলকাতার এয়ার্ডারমেজার নারিন্ড ...
4,rec_05_audio_0,"বাসু, তারলাকে আবার পুনরায় আটডাডে তেমাসি, বাবন..."


In [ ]:
df1

,External_ID,contents
0,rec_04_audio_44,"আমার বাইরা থাকতে, আমার বাইডে তবার শেফটায়, পৃথ..."
1,rec_14_audio_55,এর কিনামে সৌরতাহিত্যেও টেস্টেমেদের খেল চীনিয়া...
2,rec_22_audio_69,"সুরুজনীক করছেন, ঢাকতি দাঁপুরদরা, ঢাকতি দিনীকপে..."
3,rec_19_audio_24,আমি উদালি যুটি কলকাতার এয়ার্ডারমেজার নারিন্ড ...
4,rec_05_audio_0,"বাসু, তারলাকে আবার পুনরায় আটডাডে তেমাসি, বাবন..."
...,...,...
1103,rec_04_audio_6,আর আমি এর না-হ-বি-মি-ডাজে মনোকরি ঝাক মনো হয়ে-...
1104,rec_16_audio_7,"ডেভিটি নেকটি বিন্দো-তানে হয় তোলে ব্যাটসমেন, মা..."
1105,rec_04_audio_65,আফা! আফ! তুই বালাস-ও-আফ! বালাস-ই-এ-গুজী-কোতো ক...
1106,rec_23_audio_40,"বালোকেও কিছু করতে পারে রান পায়, বাই পেয়েলোয়..."


In [ ]:
df1.to_csv("submission1.csv",index=False)

In [ ]:
import os
test_path = "/kaggle/input/narsingdi-train-test/narsingdi/narsingdi_test"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)

    #sanity check
    if len(prediction)==0:
        prediction = "<>"

    sentences.append(prediction)



  0%|          | 0/277 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 277/277 [15:46<00:00,  3.42s/it]


In [ ]:
df2 = pd.DataFrame({"External_ID":ids,"contents":sentences})
df2.head()

,External_ID,contents
0,rec_25_audio_43,"কাঁসামুরিস, পরে আবার ডিন, আবার নুরুজলশেয়ের মস..."
1,rec_25_audio_53,"মরিজ টাবার আলাদা বাটম, কাছা মরিজ না সুকলা মরিজ..."
2,rec_23_audio_76,"স্বপ্নতঃ সবেদ, আমরা দুই টামেজ্যেজিদ বহে ডেভাইব..."
3,rec_26_audio_19,আঙ্গাপার জমায় জমায় দেয়ে আপাত্য আগেমাশে আমুগ...
4,rec_25_audio_71,"নয়েছে, ও ডাইলের মধ্যে যামনে দেই আম্রীয়।"


In [ ]:
df2

,External_ID,contents
0,rec_25_audio_43,"কাঁসামুরিস, পরে আবার ডিন, আবার নুরুজলশেয়ের মস..."
1,rec_25_audio_53,"মরিজ টাবার আলাদা বাটম, কাছা মরিজ না সুকলা মরিজ..."
2,rec_23_audio_76,"স্বপ্নতঃ সবেদ, আমরা দুই টামেজ্যেজিদ বহে ডেভাইব..."
3,rec_26_audio_19,আঙ্গাপার জমায় জমায় দেয়ে আপাত্য আগেমাশে আমুগ...
4,rec_25_audio_71,"নয়েছে, ও ডাইলের মধ্যে যামনে দেই আম্রীয়।"
...,...,...
272,rec_24_audio_62,বাচ্চ আকারে যুদ্ধে এলিই সিডিশিয়া এত্যা গানারি...
273,rec_23_audio_71,"ডেডিকেটেড পার্সন খুব ভালো, হাইপেলে ডাক ভেলো, এ..."
274,rec_23_audio_72,"চাই মাওয়ালনের ক্ত্রিমতে ভালো ক্ষোভাবক, ভালো খ..."
275,rec_25_audio_86,"মানে এড়ানিয়া, লজ্জে পাওয়া বা স্বরমের্ত এমন ..."


In [ ]:
df2.to_csv("submission2.csv",index=False)

In [ ]:
values1 = df1[['External_ID','contents']]
values2 = df2[['External_ID','contents']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,contents
0,rec_04_audio_44,"আমার বাইরা থাকতে, আমার বাইডে তবার শেফটায়, পৃথ..."
1,rec_14_audio_55,এর কিনামে সৌরতাহিত্যেও টেস্টেমেদের খেল চীনিয়া...
2,rec_22_audio_69,"সুরুজনীক করছেন, ঢাকতি দাঁপুরদরা, ঢাকতি দিনীকপে..."
3,rec_19_audio_24,আমি উদালি যুটি কলকাতার এয়ার্ডারমেজার নারিন্ড ...
4,rec_05_audio_0,"বাসু, তারলাকে আবার পুনরায় আটডাডে তেমাসি, বাবন..."
...,...,...
272,rec_24_audio_62,বাচ্চ আকারে যুদ্ধে এলিই সিডিশিয়া এত্যা গানারি...
273,rec_23_audio_71,"ডেডিকেটেড পার্সন খুব ভালো, হাইপেলে ডাক ভেলো, এ..."
274,rec_23_audio_72,"চাই মাওয়ালনের ক্ত্রিমতে ভালো ক্ষোভাবক, ভালো খ..."
275,rec_25_audio_86,"মানে এড়ানিয়া, লজ্জে পাওয়া বা স্বরমের্ত এমন ..."


In [ ]:
join.to_csv("3.5-narsingdi-banglaWhisper-inferences.csv")

In [ ]:
join.to_excel("3.5-narsingdi-banglaWhisper-inferences.xlsx")

In [ ]:
print('done')

done
